<a href="https://colab.research.google.com/github/Sarvagya4/LangChain/blob/main/Langchain9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain==0.1.4
!pip install -q langchain-openai==0.0.5
!pip install langchainhub==0.1.14
#!pip install --upgrade -q tiktoken==0.5.2
!pip install -q datasets==2.16.1
!pip install -q chromadb==0.4.22

In [ ]:
!pip install -q openai==1.10.0


In [ ]:
from datasets import load_dataset

data = load_dataset("keivalya/MedQuad-MedicalQnADataset", split='train')


In [ ]:
data = data.to_pandas()
data.head(10)


In [ ]:

from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma


In [ ]:
df_loader = DataFrameLoader(data, page_content_column="Answer")

In [ ]:
df_document = df_loader.load()
display(df_document[:2])

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1250, chunk_overlap=100)
texts = text_splitter.split_documents(df_document)

In [ ]:
first_doc = texts[1]
print(first_doc.page_content)

In [ ]:
from getpass import getpass
OPENAI_API_KEY = getpass("OpenAI API Key: ")

In [ ]:
from langchain_openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'
#model_name = 'text-embedding-3-small'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [ ]:
directory_cdb = '/content/drive/MyDrive/chromadb'
chroma_db = Chroma.from_documents(
    df_document, embed, persist_directory=directory_cdb
)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain_openai import OpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

llm=OpenAI(openai_api_key=OPENAI_API_KEY,
           temperature=0.0)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=4, #Number of messages stored in memory
    return_messages=True #Must return the messages in the response.
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=chroma_db.as_retriever()
)

In [ ]:
qa.run("What is the main symptom of LCM?")

In [ ]:
from langchain.agents import Tool, AgentExecutor

#Defining the list of tool objects to be used by LangChain.
tools = [
    Tool(
        name='Medical KB',
        func=qa.run,
        description=(
            'use this tool when answering medical knowledge queries to get '
            'more information about the topic'
        )
    )
]


In [ ]:
from langchain.agents import initialize_agent, create_react_agent
from langchain import hub

prompt = hub.pull("hwchase17/react-chat")
agent = create_react_agent(
    #agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    prompt=prompt
    #verbose=True,
    #max_iterations=3,
    #early_stopping_method='generate',
    #memory=conversational_memory
)

In [ ]:

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               memory=conversational_memory,
                               max_iterations=30,
                               max_execution_time=600,
                               #early_stopping_method='generate',
                               handle_parsing_errors=True
                               )

In [ ]:
agent_executor.invoke({"input": "Give me the area of square of 2x2"})


In [ ]:
agent_executor.invoke({"input": "Do you know who is Clark Kent?"})

In [ ]:
 agent_executor.memory.clear()

In [ ]:
agent_executor.invoke({"input": """I have a patient that can have Botulism,
how can I confirm the diagnose?"""})


In [ ]:
agent_executor.invoke({"input": "Is this an important illness?"})